<a href="https://colab.research.google.com/github/juliantsang1/capstone/blob/main/Ensemble_Playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
sample_input = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/w210_capstone/ensemble_playground/model_output (2).csv', sep=';')
sample_input.head()

Unnamed: 0  ...              image_id_3_bbox
0           0  ...                          NaN
1           1  ...   0.6759,0.741,0.5301,0.3334
2           2  ...                          NaN
3           3  ...  0.3945,0.5594,0.1602,0.1392
4           4  ...  0.3517,0.9074,0.4107,0.1808

[5 rows x 26 columns]

In [14]:
sample_input[~sample_input.image_id_1_species_name.isnull()][:5]

Unnamed: 0  model_output_id  ...  image_id_2_bbox image_id_3_bbox
1206        1206             3751  ...              NaN             NaN
1208        1208             3752  ...              NaN             NaN
1209        1209             3753  ...              NaN             NaN
1210        1210             3754  ...              NaN             NaN
1211        1211             3755  ...              NaN             NaN

[5 rows x 26 columns]

In [ ]:
sample_input['model_id'].value_counts()

5    1204
3    1204
4    1204
Name: model_id, dtype: int64

In [13]:
## for any given event, there are multiple model_ids
## model_id3 = yolo_species
## model_id4 = efficientnet_species
## model_id5 = megadetector_counts
sample_input[sample_input['image_group_id'] == 'SSWI000000019125218']

Unnamed: 0  ...  image_id_3_bbox
0              0  ...              NaN
1206        1206  ...              NaN
2436        2436  ...              NaN

[3 rows x 26 columns]

### load test data

In [ ]:
test_data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/w210_capstone/yolo_splits_3/yolo_splits3/test/test_labels.csv')
test_data[test_data.TRIGGER_ID == 'SSWI000000019125218']

TRIGGER_ID CLASS_SPECIES  Total CLASS_SPECIES_RESTATED
588  SSWI000000019125218        Coyote      1                 coyote

In [ ]:
#YOLO Species Logic
# Species
#
# Events with multiple images of same class will be labeled the majority class
#   For event to be labeled Blank, all images must be blank
# Events with all different labels will get labelled with highest confidence score
#
#Counts
# Events with multiple images of same class- we will take the max of the majority class
# Events with all different labels will get labelled with the count of highest conf score label

"""
output:
event level: event_id, species class name, count, bounding boxes, top3 dict (species name: confidence)
scale yolo top1 by 1.5x
"""

In [ ]:
"""
output: format:

  { event_id: ######
      {
        species_class: 0
        species_name: deer
        count: 1
        bbox: [
          [x, y, w, h]
        ]
        conf: 0.79
        top3: {
          deer: 0.79,
          elk: 0.4
          fox: 0.1
        }
      }

  }
"""

## dataframe method

In [4]:
test_df = sample_input[sample_input['image_group_id'] == 'SSWI000000019125218']
test_df

Unnamed: 0  ...  image_id_3_bbox
0              0  ...              NaN
1206        1206  ...              NaN
2436        2436  ...              NaN

[3 rows x 26 columns]

In [5]:
df_model_id3 = test_df[test_df.model_id == 3]
df_model_id4 = test_df[test_df.model_id == 4]
df_model_id5 = test_df[test_df.model_id == 5]

In [9]:
"""
in any given event, there are 3 images with predictions
for each image, return a dictionary of species and confidence scores

ex:
{'bear': '0.15524305403232574',
 'deer': '0.148423969745636',
 'elk': '0.14720852673053741',
 'foxgray_foxred': '0.0765981450676918',
 'turkey': '0.32857051491737366'}
"""

import math
def create_species_conf_dict(x, y):
  if isinstance(x, float):
    pass
  else:
    species_list = list(x.split(","))
    conf_list = list(y.split(","))
    return dict(zip(species_list, conf_list))


df_model_id3['img1_species_conf_dict'] = df_model_id3.apply(lambda x: create_species_conf_dict(x.image_id_1_species_name, x.image_id_1_conf), axis=1)
df_model_id3['img2_species_conf_dict'] = df_model_id3.apply(lambda x: create_species_conf_dict(x.image_id_2_species_name, x.image_id_2_conf), axis=1)
df_model_id3['img3_species_conf_dict'] = df_model_id3.apply(lambda x: create_species_conf_dict(x.image_id_3_species_name, x.image_id_3_conf), axis=1)

df_model_id4['img1_species_conf_dict'] = df_model_id4.apply(lambda x: create_species_conf_dict(x.image_id_1_species_name, x.image_id_1_conf), axis=1)
df_model_id4['img2_species_conf_dict'] = df_model_id4.apply(lambda x: create_species_conf_dict(x.image_id_2_species_name, x.image_id_2_conf), axis=1)
df_model_id4['img3_species_conf_dict'] = df_model_id4.apply(lambda x: create_species_conf_dict(x.image_id_3_species_name, x.image_id_3_conf), axis=1)

# df_model_id4['img1_species_conf_dict'][1206]

In [10]:
"""
since there are 3 images in an event, each image has a dictionary
for each event, return a combined dictionary of species and list of confidence scores

ex:
{'bear': ['0.15524305403232574', '0.16207468509674072'],
 'coyote': ['0.04405967891216278', '0.050556182861328125'],
 'deer': ['0.148423969745636', '0.2770918905735016'],
 'elk': ['0.14720852673053741', '0.022372109815478325'],
 'foxgray_foxred': ['0.0765981450676918', '0.02164595201611519'],
 'raccoon': ['0.064368836581707'],
 'turkey': ['0.32857051491737366', '0.8158956170082092', '0.31233787536621094'],
 'wolf': ['0.06660380959510803']}
"""

def merge_species_conf_dict(x, y, z):
  dict_list = [x,y,z]
  #print(dict_list)
  preds_dict = {}

  for item in dict_list:
    if item is None:
      pass
    else:
      for key, value in item.items():
        if key in preds_dict:
          preds_dict[key].append(value)
        else:
          preds_dict[key] = [value]
      
  return preds_dict

df_model_id3['consol_dict'] = df_model_id3.apply(lambda x: merge_species_conf_dict(x.img1_species_conf_dict, x.img2_species_conf_dict, x.img3_species_conf_dict), axis=1)
df_model_id4['consol_dict'] = df_model_id4.apply(lambda x: merge_species_conf_dict(x.img1_species_conf_dict, x.img2_species_conf_dict, x.img3_species_conf_dict), axis=1)
#df_model_id4['consol_dict'][1206]

In [ ]:
df_model_id3['consol_dict'][2436]

{'deer': ['0.36581435799598694']}

In [11]:
### get top 5 for each event
"""
for each event, consolidate the combined dictionary to output a topk result by confidence scores

ex:
k = 5
{'bear': '0.16207468509674072',
 'deer': '0.2770918905735016',
 'elk': '0.14720852673053741',
 'foxgray_foxred': '0.0765981450676918',
 'turkey': '0.8158956170082092'}
"""

def get_topk(x, k):
  topk_event_dict = {}
  top_ind = sorted(x, key=x.get, reverse=True)[:k]
  conf_scores = {}
  for i in top_ind:
    conf_scores[i] = max(x[i])
  
  return conf_scores

df_model_id3['topk_conf'] = df_model_id3.apply(lambda x: get_topk(x.consol_dict, 5), axis=1)
df_model_id4['topk_conf'] = df_model_id4.apply(lambda x: get_topk(x.consol_dict, 5), axis=1)
#df_model_id4['topk_conf'][1206]

In [ ]:
df_model_id3['topk_conf'][2436]

{'deer': '0.36581435799598694'}

In [15]:
"""
output final prediction based on highest confidence: class and corresponding confidence score
"""

def output_final_pred_species(x):
  top_ind = sorted(x, key=x.get, reverse=True)[:1]
  return top_ind[0]

def output_final_pred_conf(x):
  top_ind = sorted(x, key=x.get, reverse=True)[:1]
  return x[top_ind[0]]

df_model_id4['final_pred'] = df_model_id4.apply(lambda x: output_final_pred_species(x.topk_conf), axis=1)
df_model_id4['final_pred_conf'] = df_model_id4.apply(lambda x: output_final_pred_conf(x.topk_conf), axis=1)

df_model_id3['final_pred'] = df_model_id3.apply(lambda x: output_final_pred_species(x.topk_conf), axis=1)
df_model_id3['final_pred_conf'] = df_model_id3.apply(lambda x: output_final_pred_conf(x.topk_conf), axis=1)

df_model_id4

Unnamed: 0  model_output_id  ...  final_pred     final_pred_conf
1206        1206             3751  ...      turkey  0.8158956170082092

[1 rows x 33 columns]

In [16]:
df_model_id3

Unnamed: 0  model_output_id  ...  final_pred      final_pred_conf
2436        2436             2501  ...        deer  0.36581435799598694

[1 rows x 33 columns]

In [28]:
"""
merge model_id3 and model_id4 at the event level
the final prediction dictionaries from both models can be on the same row
"""
df_merge = pd.merge(df_model_id3, df_model_id4, how='inner', on="image_group_id", suffixes=('_3', '_4'))
df_merge = pd.merge(df_merge, df_model_id5, how="inner", on="image_group_id")
df_merge

Unnamed: 0_3  ...  image_id_3_bbox
0          2436  ...              NaN

[1 rows x 90 columns]

In [29]:
df_merge = df_merge[['image_group_id', 'image_id_1_bbox_3', 'image_id_2_bbox_3', 'image_id_3_bbox_3', 'topk_conf_3', 'final_pred_3', 'final_pred_conf_3',
                     'image_id_1_bbox_4', 'image_id_2_bbox_4', 'image_id_3_bbox_4', 'topk_conf_4', 'final_pred_4', 'final_pred_conf_4',
                     'image_id_1_bbox', 'image_id_2_bbox', 'image_id_3_bbox']]
df_merge = df_merge.rename(columns={'image_id_1_bbox':'image_id_1_bbox_5', 'image_id_2_bbox':'image_id_2_bbox_5', 'image_id_3_bbox':'image_id_3_bbox_5'})        
df_merge

image_group_id  ... image_id_3_bbox_5
0  SSWI000000019125218  ...               NaN

[1 rows x 16 columns]

In [ ]:
# """
# cleanup columns
# """
# df_merge = df_merge.drop(columns=['Unnamed: 0_3','model_output_id_3', 'image_id_1_3', 'image_id_1_species_name_3', 'image_id_1_conf_3', 'image_id_1_count_3',
#                        'image_id_1_blank_3', 'image_id_1_detectable_3', 'image_id_2_3', 'image_id_2_species_name_3', 'image_id_2_conf_3',
#                        'image_id_2_count_3', 'image_id_2_blank_3', 'image_id_2_detectable_3', 'image_id_3_3', 'image_id_3_species_name_3',
#                        'image_id_3_conf_3', 'image_id_3_count_3', 'image_id_3_blank_3', 'image_id_3_detectable_3', 'load_date_3', 'img1_species_conf_dict_3',
#                        'img2_species_conf_dict_3', 'img3_species_conf_dict_3', 'consol_dict_3',
#                        'Unnamed: 0_4', 'model_output_id_4', 'image_id_1_4', 'image_id_1_species_name_4', 'image_id_1_conf_4', 'image_id_1_count_4',
#                        'image_id_1_blank_4', 'image_id_1_detectable_4', 'image_id_2_4', 'image_id_2_species_name_4', 'image_id_2_conf_4',
#                        'image_id_2_count_4', 'image_id_2_blank_4', 'image_id_2_detectable_4', 'image_id_3_4', 'image_id_3_species_name_4',
#                        'image_id_3_conf_4', 'image_id_3_count_4', 'image_id_3_blank_4', 'image_id_3_detectable_4', 'load_date_4', 'img1_species_conf_dict_4',
#                        'img2_species_conf_dict_4', 'img3_species_conf_dict_4', 'consol_dict_4'])
# df_merge

In [30]:
"""
before we combine scores, we should weight the predictions between model_id3 and model_id4
to slightly bias the scores towards model_id3

80/67 = 1.2

multiply topk_conf_3 by 1.2
divide topk_conf_4 by 1.2
"""

def scale_model_id3(x):
  ## x is a dictionary
  return {key: float(value) * 1.2 for key, value in x.items()}

def scale_model_id4(x):
  ## x is a dictionary
  return {key: float(value) / 1.2 for key, value in x.items()}

df_merge['topk_conf_3_scaled'] = df_merge.apply(lambda x: scale_model_id3(x.topk_conf_3), axis=1)
df_merge['topk_conf_4_scaled'] = df_merge.apply(lambda x: scale_model_id4(x.topk_conf_4), axis=1)

In [31]:
"""
output final topk dictionary of species predictions and their confidence scores
"""

def combine_topk_conf(x, y):
  dict_list = [x,y]
  #print(dict_list)
  preds_dict = {}

  for item in dict_list:
    if item is None:
      pass
    else:
      for key, value in item.items():
        if key in preds_dict:
          preds_dict[key].append(value)
        else:
          preds_dict[key] = [value]
      
  return preds_dict

def get_final_topk(x, k):
  topk_event_dict = {}
  top_ind = sorted(x, key=x.get, reverse=True)[:k]
  conf_scores = {}
  for i in top_ind:
    conf_scores[i] = max(x[i])
  
  return conf_scores

df_merge['event_final_topk_conf'] = df_merge.apply(lambda x: get_final_topk(combine_topk_conf(x.topk_conf_3_scaled, x.topk_conf_4_scaled),3), axis=1)
df_merge['event_final_topk_conf'][0]

{'bear': 0.13506223758061728,
 'deer': 0.4389772295951843,
 'turkey': 0.6799130141735077}

In [32]:
## confirm top3
print("{}\n{}".format(df_merge['topk_conf_4_scaled'][0], df_merge['topk_conf_3_scaled'][0]))

{'turkey': 0.6799130141735077, 'bear': 0.13506223758061728, 'deer': 0.23090990881125134, 'elk': 0.12267377227544785, 'foxgray_foxred': 0.06383178755640984}
{'deer': 0.4389772295951843}


In [33]:
"""
output final event prediction and conf score
"""

df_merge['event_final_pred'] = df_merge.apply(lambda x: output_final_pred_species(x.event_final_topk_conf), axis=1)
df_merge['event_final_pred_conf'] = df_merge.apply(lambda x: output_final_pred_conf(x.event_final_topk_conf), axis=1)
df_merge

image_group_id  ... event_final_pred_conf
0  SSWI000000019125218  ...              0.679913

[1 rows x 21 columns]

to-do: scale model_id3 conf by 1.5 <br>
determine best bounding box

if model3, go with model3 bbox <br>
if model4, go with model5 (MD) bbox

event level: event_id, species class name, count, bounding boxes, top3 dict (species name: confidence)

In [ ]:
def find_bbox(class_name, conf_score, model3_conf_dict, model4_conf_dict):
  key, value = class_name, conf_score
  if key in model3_conf_dict and value == model3_conf_dict[key]:
    return 

## dictionary method

In [ ]:
event_dict = {}

for index, row in test_df.iterrows():

  event_id = row['image_group_id']

  model_id = row['model_id']
  img1_result_class = row['image_id_1_species_name']
  img1_result_conf = row['image_id_1_conf']
  img1_result_count = row['image_id_1_count']
  img1_bbox = row['image_id_1_bbox']

  img2_result_class = row['image_id_2_species_name']
  img2_result_conf = row['image_id_2_conf']
  img2_result_count = row['image_id_2_count']
  img2_bbox = row['image_id_2_bbox']

  img3_result_class = row['image_id_3_species_name']
  img3_result_conf = row['image_id_3_conf']
  img3_result_count = row['image_id_3_count'] 
  img3_bbox = row['image_id_3_bbox']

  result_dict = {
        "model_id": model_id,
        "img1": {
            "pred_class": img1_result_class,
            "pred_conf": img1_result_conf,
            "pred_count": img1_result_count,
            "pred_bbox": img1_bbox
        },
        "img2": {
            "pred_class": img2_result_class,
            "pred_conf": img2_result_conf,
            "pred_count": img2_result_count,
            "pred_bbox": img2_bbox            
        },
        "img3": {
            "pred_class": img3_result_class,
            "pred_conf": img3_result_conf,
            "pred_count": img3_result_count,
            "pred_bbox": img3_bbox            
        }
    }
  

  if event_id in event_dict:
    event_dict[event_id].append(result_dict)
  else:
    event_dict[event_id] = [result_dict]

event_dict

{'SSWI000000019125218': [{'img1': {'pred_bbox': nan,
    'pred_class': nan,
    'pred_conf': nan,
    'pred_count': 0},
   'img2': {'pred_bbox': '0.8788,0.6067,0.2065,0.1463',
    'pred_class': nan,
    'pred_conf': nan,
    'pred_count': 1},
   'img3': {'pred_bbox': nan,
    'pred_class': nan,
    'pred_conf': nan,
    'pred_count': 0},
   'model_id': 5},
  {'img1': {'pred_bbox': nan,
    'pred_class': 'turkey,bear,deer,elk,foxgray_foxred',
    'pred_conf': '0.32857051491737366,0.15524305403232574,0.148423969745636,0.14720852673053741,0.0765981450676918',
    'pred_count': 0},
   'img2': {'pred_bbox': nan,
    'pred_class': 'turkey,wolf,coyote,elk,foxgray_foxred',
    'pred_conf': '0.8158956170082092,0.06660380959510803,0.04405967891216278,0.022372109815478325,0.02164595201611519',
    'pred_count': 0},
   'img3': {'pred_bbox': nan,
    'pred_class': 'turkey,deer,bear,raccoon,coyote',
    'pred_conf': '0.31233787536621094,0.2770918905735016,0.16207468509674072,0.064368836581707,0.0505

#### playground

In [ ]:
import pandas as pd
model_results = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/w210_capstone/JSON_outputs/phase2_efficientnetb5_classifications_exclude_Wolf.json')
df = model_results['phase2_classification_results'].apply(pd.Series)
df['event_id'] = df['id'].str.split('.jpg').str[0]
df['event_id'] = df['event_id'].str[:-1]

In [ ]:
## for index, row in df.iloc[:5].iterrows():
preds_dict = {}
for index, row in df.iloc[:5].iterrows():
  event_id = str(row['event_id'])
  conf_dict = row['conf_dict']
  
  result_dict = {
        "conf_dict": conf_dict
    }
  
  if event_id in preds_dict:
    preds_dict[event_id].append(result_dict)
  else:
    preds_dict[event_id] = [result_dict]

preds_dict

{'SSWI000000018696587': [{'conf_dict': {'0': [0.984815597534179],
    '1': [0.0006901166634630001],
    '5': [0.000570110452827],
    '6': [0.000864675734192],
    '7': [0.012593532912433002]}},
  {'conf_dict': {'0': [0.99616539478302],
    '1': [0.00021788691810700002],
    '3': [0.000203496572794],
    '6': [0.00040631039883],
    '7': [0.0026421239599580004]}},
  {'conf_dict': {'0': [0.98821347951889],
    '2': [0.000333709089318],
    '5': [0.0008325020316980001],
    '6': [0.0016884513897820002],
    '7': [0.008449412882328]}}],
 'SSWI000000018741710': [{'conf_dict': {'0': [0.9460017681121821],
    '2': [0.000399870128603],
    '5': [0.001346027012914],
    '6': [0.022270414978265003],
    '7': [0.02964262664318]}},
  {'conf_dict': {'0': [0.27309530973434404],
    '2': [0.018853003159165],
    '5': [0.255657702684402],
    '6': [0.17894247174263],
    '7': [0.257040947675704]}}]}

In [ ]:
consol_dict = {}
for item, list_dict in preds_dict.items():
  itr = 0
  length = len(list_dict)
  event_id = item
  while itr < length:
    for k,v in list_dict[itr].items():
      #print(k, v)
      if event_id in consol_dict:
        consol_dict[event_id].append(v)
      else:
        consol_dict[event_id] = [v]
      #consol_dict.setdefault(k, []).append(v)
      itr +=1

consol_dict

{'SSWI000000018696587': [{'0': [0.984815597534179],
   '1': [0.0006901166634630001],
   '5': [0.000570110452827],
   '6': [0.000864675734192],
   '7': [0.012593532912433002]},
  {'0': [0.99616539478302],
   '1': [0.00021788691810700002],
   '3': [0.000203496572794],
   '6': [0.00040631039883],
   '7': [0.0026421239599580004]},
  {'0': [0.98821347951889],
   '2': [0.000333709089318],
   '5': [0.0008325020316980001],
   '6': [0.0016884513897820002],
   '7': [0.008449412882328]}],
 'SSWI000000018741710': [{'0': [0.9460017681121821],
   '2': [0.000399870128603],
   '5': [0.001346027012914],
   '6': [0.022270414978265003],
   '7': [0.02964262664318]},
  {'0': [0.27309530973434404],
   '2': [0.018853003159165],
   '5': [0.255657702684402],
   '6': [0.17894247174263],
   '7': [0.257040947675704]}]}

In [ ]:
outer_dict = {}
for key, value in consol_dict.items():
  intermed_dict = {}
  for sub_dict in value:
    for k,v in sub_dict.items():
      if k in intermed_dict:
        intermed_dict[k].append(v[0])
      else:
        intermed_dict[k] = [v[0]]
      outer_dict[key] = intermed_dict

outer_dict

{'SSWI000000018696587': {'0': [0.984815597534179,
   0.99616539478302,
   0.98821347951889],
  '1': [0.0006901166634630001, 0.00021788691810700002],
  '2': [0.000333709089318],
  '3': [0.000203496572794],
  '5': [0.000570110452827, 0.0008325020316980001],
  '6': [0.000864675734192, 0.00040631039883, 0.0016884513897820002],
  '7': [0.012593532912433002, 0.0026421239599580004, 0.008449412882328]},
 'SSWI000000018741710': {'0': [0.9460017681121821, 0.27309530973434404],
  '2': [0.000399870128603, 0.018853003159165],
  '5': [0.001346027012914, 0.255657702684402],
  '6': [0.022270414978265003, 0.17894247174263],
  '7': [0.02964262664318, 0.257040947675704]}}

In [ ]:
### get top 5 for each event
final_top5_event_dict = {}
for key, value in outer_dict.items():
  top_ind = sorted(value, key=value.get, reverse=True)[:5]
  conf_scores = {}
  for i in top_ind:
    conf_scores[i] = max(value[i])
  
  final_top5_event_dict[key] = conf_scores

final_top5_event_dict

{'SSWI000000018696587': {'0': 0.99616539478302,
  '1': 0.0006901166634630001,
  '5': 0.0008325020316980001,
  '6': 0.0016884513897820002,
  '7': 0.012593532912433002},
 'SSWI000000018741710': {'0': 0.9460017681121821,
  '2': 0.018853003159165,
  '5': 0.255657702684402,
  '6': 0.17894247174263,
  '7': 0.257040947675704}}

### continuing

In [ ]:
## consolidate predictions from models 3 and 4
"""
'SSWI000000018696587': [{'0': [0.984815597534179],
   '1': [0.0006901166634630001],
   '5': [0.000570110452827],
   '6': [0.000864675734192],
   '7': [0.012593532912433002]},
  {'0': [0.99616539478302],
   '1': [0.00021788691810700002],
   '3': [0.000203496572794],
   '6': [0.00040631039883],
   '7': [0.0026421239599580004]},
  {'0': [0.98821347951889],
   '2': [0.000333709089318],
   '5': [0.0008325020316980001],
   '6': [0.0016884513897820002],
   '7': [0.008449412882328]}]
"""

consol_dict = {}

for item, list_dict in event_dict.items():
  itr = 0
  intermediate_dict = list_dict
  #print(intermediate_dict)
  event_id = item
  length = len(intermediate_dict)
  while itr < length:
    for k,v in intermediate_dict[itr].items():
      print(k,v)
      if k=="model_id" and v==5:
        continue
      else:
        if 'img' in k:
          if event_id in consol_dict:
            consol_dict[event_id].append(v)
          else:
            consol_dict[event_id] = [v]

    itr +=1


consol_dict

  # itr = 0
  # length = len(list_dict)
  # print(length)
  # event_id = item
  # while itr < length:
  #   for k,v in list_dict[itr].items():
  #     #print(k, v)
  #     if event_id in consol_dict:
  #       consol_dict[event_id].append(v)
  #     else:
  #       consol_dict[event_id] = [v]
  #     #consol_dict.setdefault(k, []).append(v)
  #     itr +=1


model_id 5
img1 {'pred_class': nan, 'pred_conf': nan, 'pred_count': 0, 'pred_bbox': nan}
img2 {'pred_class': nan, 'pred_conf': nan, 'pred_count': 1, 'pred_bbox': '0.8788,0.6067,0.2065,0.1463'}
img3 {'pred_class': nan, 'pred_conf': nan, 'pred_count': 0, 'pred_bbox': nan}
model_id 4
img1 {'pred_class': 'turkey,bear,deer,elk,foxgray_foxred', 'pred_conf': '0.32857051491737366,0.15524305403232574,0.148423969745636,0.14720852673053741,0.0765981450676918', 'pred_count': 0, 'pred_bbox': nan}
img2 {'pred_class': 'turkey,wolf,coyote,elk,foxgray_foxred', 'pred_conf': '0.8158956170082092,0.06660380959510803,0.04405967891216278,0.022372109815478325,0.02164595201611519', 'pred_count': 0, 'pred_bbox': nan}
img3 {'pred_class': 'turkey,deer,bear,raccoon,coyote', 'pred_conf': '0.31233787536621094,0.2770918905735016,0.16207468509674072,0.064368836581707,0.050556182861328125', 'pred_count': 0, 'pred_bbox': nan}
model_id 3
img1 {'pred_class': nan, 'pred_conf': nan, 'pred_count': 0, 'pred_bbox': nan}
img2 {

{'SSWI000000019125218': [{'pred_bbox': nan,
   'pred_class': nan,
   'pred_conf': nan,
   'pred_count': 0},
  {'pred_bbox': '0.8788,0.6067,0.2065,0.1463',
   'pred_class': nan,
   'pred_conf': nan,
   'pred_count': 1},
  {'pred_bbox': nan, 'pred_class': nan, 'pred_conf': nan, 'pred_count': 0},
  {'pred_bbox': nan,
   'pred_class': 'turkey,bear,deer,elk,foxgray_foxred',
   'pred_conf': '0.32857051491737366,0.15524305403232574,0.148423969745636,0.14720852673053741,0.0765981450676918',
   'pred_count': 0},
  {'pred_bbox': nan,
   'pred_class': 'turkey,wolf,coyote,elk,foxgray_foxred',
   'pred_conf': '0.8158956170082092,0.06660380959510803,0.04405967891216278,0.022372109815478325,0.02164595201611519',
   'pred_count': 0},
  {'pred_bbox': nan,
   'pred_class': 'turkey,deer,bear,raccoon,coyote',
   'pred_conf': '0.31233787536621094,0.2770918905735016,0.16207468509674072,0.064368836581707,0.050556182861328125',
   'pred_count': 0},
  {'pred_bbox': nan, 'pred_class': nan, 'pred_conf': nan, 'pr